<a href="https://colab.research.google.com/github/ChadDelany/Drought_Prediction/blob/main/04_Modeling_RAPIDS_allFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Drought Prediction: Modeling**

This notebook was run on Google Colab Pro+.  The RAPIDS system was used to access GPU processing.  Previous attempts to run the models using Pandas with CPUs took 24+ hours to run and often crashed due to exceeding existing resources.  The RAPIDS GPU processing allowed models to run usually within 5 minutes and at a maximum of 15 minutes.  Setting up RAPIDS to run on Google Colab Pro+ takes between 15 minutes to 1 hour depending on resource availability on Google Colab Pro+.

ALL MODELS INITIALLY RUN WITH ALL AVAILABLE VARIABLES TO DETERMINE INITIAL MODEL PERFORMANCE.

# Environment Sanity Check #

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4, P4, or P100.

In [ ]:
!nvidia-smi

Wed Oct 26 22:30:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    45W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Setup:
This notebook was built on RAPIDS 0.13 stable and is based on this [DataCamp Tutorial](https://www.datacamp.com/community/tutorials/xgboost-in-python).  tested and working on 0.19 stable.

#Setup:
Set up script installs
1. Updates gcc in Colab
1. Installs Conda
1. Install RAPIDS' current stable version of its libraries, as well as some external libraries including:
  1. cuDF
  1. cuML
  1. cuGraph
  1. cuSpatial
  1. cuSignal
  1. BlazingSQL
  1. xgboost
1. Copy RAPIDS .so files into current working directory, a neccessary workaround for RAPIDS+Colab integration.


In [ ]:
!pip install pynvml

# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 2.6 MB/s 
Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 300, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 300 (delta 74), reused 99 (delta 55), pack-reused 171
Receiving objects: 100% (300/300), 87.58 KiB | 3.65 MiB/s, done.
Resolving deltas: 100% (136/136), done.
***********************************************************************
Woo! Your instance has the right kind of GPU, a A100-SXM4-40GB!
***********************************************************************



In [ ]:
# This will update the Colab environment and restart the kernel.  Don't run the next cell until you see the session crash.
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1,217 kB]
Get:13 http://ar

In [ ]:
# This will install CondaColab.  This will restart your kernel one last time.  Run this cell by itself and only run the next cell once you see the session crash.
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:19
🔁 Restarting kernel...


In [ ]:
# you can now run the rest of the cells as normal
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
# Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
!python rapidsai-csp-utils/colab/install_rapids.py stable
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'

Found existing installation: cffi 1.15.1
Uninstalling cffi-1.15.1:
  Successfully uninstalled cffi-1.15.1
Found existing installation: cryptography 37.0.4
Uninstalling cryptography-37.0.4:
  Successfully uninstalled cryptography-37.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.1/427.1 kB 9.3 MB/s eta 0:00:00
Installing RAPIDS Stable 21.12
Starting the RAPIDS install on Colab.  This will take about 15 minutes.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cudatoolkit=11.2
    - dask-sql
    - gcsfs
    - llvmlite
    - openssl
    - python=3.7
    - rapids=21.12


The following package

## Load RAPIDS libraries

In [ ]:
# RAPIDS libraries for accessing GPU processing for running models.  Instead of taking 24+ hours to run models, it only takes 15 minutes or less.
import cudf
import cuml
import cupy

import pandas as pd

import pynvml
import numpy as np


## Load additional Libraries.

In [ ]:
# Import Sklearn metrics.  The RAPIDS metrics currently appeared bugged.
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_val_score, StratifiedKFold

# For displaying model metrics in an easily readable table form.
from IPython.display import HTML, display
import tabulate

# Functions

## Model Accuracy Assessment for Regression.

In [ ]:
# Function for Model Accuracy Assessment for Regression.  Input is in Pandas because RAPIDS metrics currently appeared to be bugged.
def reg_metric(y_test, y_pred):
  # Calculation of metrics
  r2 = str(round(r2_score(y_test, y_pred), 3))
  mse = str(round(mean_squared_error(y_test, y_pred), 3))
  rmse = str(round(np.sqrt(mean_squared_error(y_test, y_pred)), 3))
  mae = str(round(mean_absolute_error(y_test, y_pred), 3))

  #create table for display
  metric = [['Metric', 'Value'],
            ['R**2:', r2],
            ['MSE:', mse],
            ['RMSE:', rmse],
            ['MAE:', mae]]
  table = tabulate.tabulate(metric, tablefmt='html')
  display(HTML(table))

  # Return metric values as strings for later display.
  return(r2, mse, rmse, mae)

## Coefficients from Regression Models.

In [ ]:
# Function to pull coefficients for regression models.
def reg_coefs(model):
  # get coefficients from RAPIDS model
  coefs = model.coef_
  coefs = coefs.to_pandas()

  # associate variable names with coefficients
  features = X_train.columns
  
  # Create Pandas Series with appropriate labels
  coefs = coefs.set_axis(features)

  return(coefs)

## Model Accuracy Assessment for Classification.

In [ ]:
# Function for calculating Classification Metrics
def class_metric(ycat_test, y_pred):
  #Calculation of metrics
  accuracy = str(np.round(cuml.metrics.accuracy.accuracy_score(ycat_test, y_pred), 3) * 100)
  roc_auc = str(np.round(cuml.metrics.roc_auc_score(ycat_test, y_pred), 3))

  cp = np.round(cuml.metrics.confusion_matrix(ycat_test, y_pred, normalize='pred'), 3) * 100
  pred_perclass = [cp[0][0].get(), cp[1][1].get(), cp[2][2].get(), cp[3][3].get(), cp[4][4].get(), cp[5][5].get()]

  cp_mean = str(np.round(np.mean(pred_perclass), 1))
  cp_std = str(np.round(np.std(pred_perclass), 1))

  print(f'Accuracy Score: {accuracy}%')
  print(f'ROC AUC: {roc_auc}')
  print(f'Mean Accuracy per Class & Standard Deviation: {cp_mean}% +/- {cp_std}%')
  print(cp)

  return(accuracy, roc_auc, cp, cp_mean, cp_std)

# Load Dataset.

In [ ]:
# Local location of the data

# Location on Windows
# local_data = 'D:\\Data_Science\\DroughtProject\\Data\\' 

# Location on Linux
# local_data = '/home/chad/Data/Drought_Prediction/' 

# Load local data into Google Colab
# from google.colab import files
# files = files.upload()

In [ ]:
# Accessing Google Drive by mounting it locally
# https://towardsdatascience.com/7-ways-to-load-external-data-into-google-colab-7ba73e7d5fc7
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Location on Google Drive
local_data = '/content/drive/MyDrive/Colab Notebooks/'

In [ ]:
# Load the dataset that contains training (meteorological variables) resampled weekly with mean, max, min
# and the soil variables that have been merged on the county 'fips' value
# This version of the file has already been scaled for the mean equal to zero and the variance to a standard deviation via StandardScaler.

tsm = cudf.read_csv(local_data + 'train_soil_stats_scaled.csv',
                        parse_dates=['date'],
                        index_col=['index'],
                        header=0)

In [ ]:
# Unmount Google Drive.
drive.flush_and_unmount()

## Select Features and Target for Models

In [ ]:
# Breaking out independent numerical variables from target variable, categorical variable ('fips'), and date.
cols = tsm.columns.tolist()
features = cols[3:]

# Separating out the features
X = tsm[features]

# Separating out the target
y = tsm[['score']]

In [ ]:
# Converting 'y' from Panda Dataframe to Panda Series to avoid conflicts with type when running RAPIDS models.
y = y['score']

# Convert X from float64 to float32 in order to utilize GPU processing instead of CPU processing.  RAPIDS currently does not support float64.
X = X.astype('float32')

# Convert y from float64 to float32 in order to utilize GPU processing instead of CPU processing.
y = y.astype('float32')

# Create target for classication models.  Drought Score was originally an integer class ranging 0 - 5.
y_cat = np.round(y,0)
y_cat = y_cat.astype(int)

## Train / Test Split, Random_State=42

In [ ]:
# Create first train/test split for regression.  Automated folds from cross-validation are consuming too many resources and taking too long to process.
# The datset is significantly skewed with many examples near 0 and few examples near 5.  The Train/Test split is therefore stratified to account for this.
X_train, X_test, y_train, y_test = cuml.model_selection.train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42, stratify=y_cat)

# Not-Even-A-Model

In [ ]:
# Using sklearn requires Pandas & not RAPIDS cudf
X_train = X_train.to_pandas()
X_test = X_test.to_pandas()
y_train = y_train.to_pandas()
y_test = y_test.to_pandas()

# Dummy Regressor to establish a baseline.
Dumb_reg = DummyRegressor(strategy='mean')
Dumb_reg.fit(X_train, y_train)
y_pred = Dumb_reg.predict(X_test)

In [ ]:
# Dummy Regresion Model assessment: R-squared, Mean Squared Error, Root MSE, Mean Absolute Error.
Dum_r2, Dum_mse, Dum_rmse, Dum_mae = reg_metric(y_test, y_pred)

Metric,Value
R**2:,-0.0
MSE:,1.497
RMSE:,1.224
MAE:,0.974


In [ ]:
# There is a bug in RAPIDS for converting Pandas Series back into cudf Series, so just rerunning the Test / Train split with the same random state.
X_train, X_test, y_train, y_test = cuml.model_selection.train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42, stratify=y_cat)
y_test = y_test.to_pandas()

# Regression Models

## Linear Regression

In [ ]:
# Linear Regression 
Lin_model = cuml.LinearRegression(fit_intercept=True, normalize=True)
Lin_model.fit(X_train, y_train)
y_pred = Lin_model.predict(X_test)

In [ ]:
# Convert RAPIDS to Pandas for Regression Metrics Calculations.
y_pred = y_pred.to_pandas()

In [ ]:
# Linear Regresion Model assessment: R-squared, Mean Squared Error, Root MSE, Mean Absolute Error.
Lin_r2, Lin_mse, Lin_rmse, Lin_mae = reg_metric(y_test, y_pred)

Metric,Value
R**2:,0.215
MSE:,1.176
RMSE:,1.084
MAE:,0.819


In [ ]:
# Looking at coefficients for the regression model to determine importance of variables within the model.  
# These results are only preliminary and need to assessed when there is less correlation between input variables.

# Function call to pull coefficients.
coefs = reg_coefs(Lin_model)

# Display sorted values of coefficients.
coefs.sort_values(ascending=False)

T2MDEW_mean         15.251828
T2MWET_max          13.392090
WS50M_MAX_mean      11.275458
aspectUnknown        8.904612
T2M_MAX_mean         6.267947
                      ...    
WS50M_MIN_mean      -6.379287
slope2              -7.124270
WS50M_RANGE_mean    -7.522495
T2MDEW_max         -13.325664
T2MWET_mean        -16.505831
Length: 85, dtype: float32

In [ ]:
# Displaying all coefficient values to understand range and variability.
# Sorted absolute values to understand total influence on model.
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(abs(coefs).sort_values(ascending=False))

T2MWET_mean         16.505831
T2MDEW_mean         15.251828
T2MWET_max          13.392090
T2MDEW_max          13.325664
WS50M_MAX_mean      11.275458
aspectUnknown        8.904612
WS50M_RANGE_mean     7.522495
slope2               7.124270
WS50M_MIN_mean       6.379287
T2M_MAX_mean         6.267947
T2MDEW_min           5.877740
T2MWET_min           5.603858
T2M_mean             3.599877
slope1               2.928799
T2M_MIN_mean         1.949099
WS10M_MAX_mean       1.752050
TS_min               1.284496
WS10M_RANGE_mean     1.275383
T2M_min              1.206666
T2M_RANGE_mean       1.119282
TS_max               0.813282
TS_mean              0.810729
T2M_max              0.748197
WS10M_MIN_mean       0.730025
PS_mean              0.716312
PS_min               0.681556
WS10M_mean           0.413662
CULT_LAND            0.386309
WS50M_mean           0.367662
QV2M_mean            0.345696
CULTRF_LAND          0.308219
SQ1                  0.140983
slope5               0.137277
QV2M_max  

In [ ]:
# The variables with the largest coefficients from the simple linear regresssion.
top_features = abs(coefs).sort_values(ascending=False)
top_features[:15]

T2MWET_mean         16.505831
T2MDEW_mean         15.251828
T2MWET_max          13.392090
T2MDEW_max          13.325664
WS50M_MAX_mean      11.275458
aspectUnknown        8.904612
WS50M_RANGE_mean     7.522495
slope2               7.124270
WS50M_MIN_mean       6.379287
T2M_MAX_mean         6.267947
T2MDEW_min           5.877740
T2MWET_min           5.603858
T2M_mean             3.599877
slope1               2.928799
T2M_MIN_mean         1.949099
dtype: float32

### Ridge Regresssion

In [ ]:
# Ridge Regression - L2 regularization
Ridge_model = cuml.Ridge(fit_intercept=True, normalize=True)
Ridge_model.fit(X_train, y_train)
y_pred = Ridge_model.predict(X_test)

In [ ]:
# Convert y_pred to Pandas for metrics calc.
y_pred = y_pred.to_pandas()

In [ ]:
# Regresion Model assessment: R-squared, Mean Squared Error, Root MSE, Mean Absolute Error.
Ridge_r2, Ridge_mse, Ridge_rmse, Ridge_mae = reg_metric(y_test, y_pred)

Metric,Value
R**2:,0.154
MSE:,1.266
RMSE:,1.125
MAE:,0.872


In [ ]:
# Pulling coefficients from the model.
coefs = reg_coefs(Ridge_model)
coefs.sort_values(ascending=False)

T2M_RANGE_mean    0.054816
T2M_RANGE_max     0.047383
T2M_MAX_max       0.042160
GRS_LAND          0.036442
T2M_MAX_mean      0.035399
                    ...   
T2MDEW_mean      -0.038325
T2MWET_min       -0.040171
T2MDEW_min       -0.041343
lon              -0.048592
lat              -0.071717
Length: 85, dtype: float32

In [ ]:
# The variables with the largest coefficients from the Ridge regresssion.
top_features = abs(coefs).sort_values(ascending=False)
top_features[:15]

lat               0.071717
T2M_RANGE_mean    0.054816
lon               0.048592
T2M_RANGE_max     0.047383
T2M_MAX_max       0.042160
T2MDEW_min        0.041343
T2MWET_min        0.040171
T2MDEW_mean       0.038325
T2MWET_mean       0.037785
GRS_LAND          0.036442
T2M_MAX_mean      0.035399
T2MDEW_max        0.035090
T2MWET_max        0.034974
T2M_RANGE_min     0.032373
PRECTOT_max       0.031478
dtype: float32

### Lasso Regression

In [ ]:
# Lasso Regression - L1 regularization
Lasso_model = cuml.Lasso(fit_intercept=True, normalize=True)
Lasso_model.fit(X_train, y_train)
y_pred = Lasso_model.predict(X_test)

In [ ]:
# Convert y_pred to Pandas for metrics calc.
y_pred = y_pred.to_pandas()

In [ ]:
# Regresion Model assessment: R-squared, Mean Squared Error, Root MSE, Mean Absolute Error.
Lasso_r2, Lasso_mse, Lasso_rmse, Lasso_mae = reg_metric(y_test, y_pred)

Metric,Value
R**2:,-0.0
MSE:,1.497
RMSE:,1.224
MAE:,0.974


In [ ]:
# Pulling coefficients from the model.
coefs = reg_coefs(Lasso_model)
coefs.sort_values(ascending=False)

PRECTOT_mean     0.0
lat              0.0
slope6           0.0
slope5           0.0
slope4           0.0
                ... 
TS_max           0.0
T2M_RANGE_max    0.0
T2M_MIN_max      0.0
T2M_MAX_max      0.0
SQ7              0.0
Length: 85, dtype: float32

In [ ]:
# The variables with the largest coefficients from the simple linear regresssion.
top_features = abs(coefs).sort_values(ascending=False)
top_features[:15]

PRECTOT_mean       0.0
lat                0.0
slope6             0.0
slope5             0.0
slope4             0.0
slope3             0.0
slope2             0.0
slope1             0.0
elevation          0.0
lon                0.0
WS50M_RANGE_min    0.0
T2M_MIN_min        0.0
WS50M_MIN_min      0.0
WS50M_MAX_min      0.0
WS50M_min          0.0
dtype: float32

### ElasticNet Regression

In [ ]:
# ElasticNet Regression - combination of L1 and L2 regularization
EN_model = cuml.ElasticNet(alpha=0.33)
EN_model.fit(X_train, y_train)
y_pred = EN_model.predict(X_test)

In [ ]:
# Convert y_pred to Pandas for metrics calc.
y_pred = y_pred.to_pandas()

In [ ]:
# Regresion Model assessment: R-squared, Mean Squared Error, Root MSE, Mean Absolute Error.
EN_r2, EN_mse, EN_rmse, EN_mae = reg_metric(y_test, y_pred)

Metric,Value
R**2:,0.074
MSE:,1.387
RMSE:,1.178
MAE:,0.929


In [ ]:
# Pulling coefficients from the model.
coefs = reg_coefs(EN_model)
coefs.sort_values(ascending=False)

T2M_RANGE_mean    0.145191
GRS_LAND          0.037953
PRECTOT_mean      0.000000
lat               0.000000
slope6            0.000000
                    ...   
TS_max            0.000000
T2M_RANGE_max     0.000000
T2M_MIN_max       0.000000
SQ7               0.000000
lon              -0.033884
Length: 85, dtype: float32

In [ ]:
# The variables with the largest coefficients from the simple linear regresssion.
top_features = abs(coefs).sort_values(ascending=False)
top_features[:15]

T2M_RANGE_mean     0.145191
GRS_LAND           0.037953
lon                0.033884
PRECTOT_mean       0.000000
lat                0.000000
slope5             0.000000
slope4             0.000000
slope3             0.000000
slope2             0.000000
slope1             0.000000
elevation          0.000000
WS50M_RANGE_min    0.000000
slope7             0.000000
WS50M_MIN_min      0.000000
WS50M_MAX_min      0.000000
dtype: float32

## Nearest Neighbors Regresssion

In [ ]:
NNreg_model = cuml.neighbors.KNeighborsRegressor()
NNreg_model.fit(X_train, y_train)
y_pred = NNreg_model.predict(X_test)

In [ ]:
# Convert y_pred to Pandas for metrics calc.
y_pred = y_pred.to_pandas()

In [ ]:
# Regresion Model assessment: R-squared, Mean Squared Error, Root MSE, Mean Absolute Error.
NN_r2, NN_mse, NN_rmse, NN_mae = reg_metric(y_test, y_pred)

Metric,Value
R**2:,0.468
MSE:,0.796
RMSE:,0.892
MAE:,0.574


### Nearest Neighbor Hyperparameter Search

In [ ]:
# Convert y_test to pandas if it has not already been converted.
y_test = y_test.to_pandas()

In [ ]:
# Search for best n_neighbors hyperparameter.  Model parameters are not causing the system to crash.
r2_list=[]
mse_list=[]
rmse_list=[]
mae_list=[]

# Searching nearest neighbors from 2 to 9.
for i in range(8):
  NNreg_model = cuml.neighbors.KNeighborsRegressor(n_neighbors=(i+2))
  NNreg_model.fit(X_train, y_train)
  y_pred = NNreg_model.predict(X_test)

  # Convert y_pred to Pandas for metrics calc.
  y_pred = y_pred.to_pandas()

  # Call custom metric function
  print(f'Nearest Neighbor: {i+2}')
  NN_r2, NN_mse, NN_rmse, NN_mae = reg_metric(y_test, y_pred)

  r2_list.append(NN_r2)
  mse_list.append(NN_mse)
  rmse_list.append(NN_rmse)
  mae_list.append(NN_mae)

Nearest Neighbor: 2


Metric,Value
R**2:,0.386
MSE:,0.92
RMSE:,0.959
MAE:,0.554


Nearest Neighbor: 3


Metric,Value
R**2:,0.439
MSE:,0.84
RMSE:,0.917
MAE:,0.558


Nearest Neighbor: 4


...

Metric,Value
R**2:,0.47
MSE:,0.793
RMSE:,0.891
MAE:,0.591


Nearest Neighbor: 8


Metric,Value
R**2:,0.468
MSE:,0.796
RMSE:,0.892
MAE:,0.6


Nearest Neighbor: 9


Metric,Value
R**2:,0.465
MSE:,0.8
RMSE:,0.895
MAE:,0.607


In [ ]:
# Print Accuracy Score for each n_neighbors
for i in range(8):
  print(f'n_neighbors={(i+2)}, Mean Absolute Error: {mae_list[i]}')

n_neighbors=2, Mean Absolute Error: 0.554
n_neighbors=3, Mean Absolute Error: 0.558
n_neighbors=4, Mean Absolute Error: 0.566
n_neighbors=5, Mean Absolute Error: 0.574
n_neighbors=6, Mean Absolute Error: 0.583
n_neighbors=7, Mean Absolute Error: 0.591
n_neighbors=8, Mean Absolute Error: 0.6
n_neighbors=9, Mean Absolute Error: 0.607


The Mean Absolute Error never goes below 0.5.  Since each integer value correlates to a Drought Score Category, having an error that exceeds 0.5 is not practically useful.

## Random Forest Regression Model

In [ ]:
# Random Forest Regression model, default max_depth=16, default n_estimators=100.
RFreg_model = cuml.ensemble.RandomForestRegressor()
RFreg_model.fit(X_train, y_train)
y_pred = RFreg_model.predict(X_test)

In [ ]:
# Convert y_pred to Pandas for metrics calc.
y_pred = y_pred.to_pandas()

In [ ]:
# Regresion Model assessment: R-squared, Mean Squared Error, Root MSE, Mean Absolute Error.
RF_r2, RF_mse, RF_rmse, RF_mae = reg_metric(y_test, y_pred)

Metric,Value
R**2:,0.486
MSE:,0.769
RMSE:,0.877
MAE:,0.648


### Random Forest Regression, max_depth = 100

In [ ]:
# Random Forest Regression model: MAX DEPTH = 100.
RFreg_model = cuml.ensemble.RandomForestRegressor(max_depth=100)
RFreg_model.fit(X_train, y_train)
y_pred = RFreg_model.predict(X_test)

In [ ]:
# Convert y_pred to Pandas for metrics calc.
y_pred = y_pred.to_pandas()

In [ ]:
# Regresion Model assessment: R-squared, Mean Squared Error, Root MSE, Mean Absolute Error.
RF_r2, RF_mse, RF_rmse, RF_mae = reg_metric(y_test, y_pred)

Metric,Value
R**2:,0.71
MSE:,0.434
RMSE:,0.659
MAE:,0.436


### Random Forest Regression, max_depth=100, n_estimators=300

In [ ]:
# Random Forest Regression model: MAX DEPTH = 100.
RFreg_model = cuml.ensemble.RandomForestRegressor(max_depth=100, n_estimators=300)
RFreg_model.fit(X_train, y_train)
y_pred = RFreg_model.predict(X_test)

In [ ]:
# Convert y_pred to Pandas for metrics calc.
y_pred = y_pred.to_pandas()

In [ ]:
# Regresion Model assessment: R-squared, Mean Squared Error, Root MSE, Mean Absolute Error.
RF_r2, RF_mse, RF_rmse, RF_mae = reg_metric(y_test, y_pred)

Metric,Value
R**2:,0.714
MSE:,0.429
RMSE:,0.655
MAE:,0.434


# **Regression Model Summary**.

In [ ]:
# Table Summarizing metrics from the different regression models.
metric = [['Metric', 'Dummy Reg', 'Linear Reg', 'Ridge Reg', 'Lasso Reg', 'ElasticNet', 'Nearest Neighbor', 'Random Forest'],
         ['R**2', Dum_r2, Lin_r2, Ridge_r2, Lasso_r2, EN_r2, NN_r2, RF_r2],
         ['MSE', Dum_mse, Lin_mse, Ridge_mse, Lasso_mse, EN_mse, NN_mse, RF_mse],
         ['RMSE', Dum_rmse, Lin_rmse, Ridge_rmse, Lasso_rmse, EN_rmse, NN_rmse, RF_rmse],
         ['MAE', Dum_mae, Lin_mae, Ridge_mae, Lasso_mae, EN_mae, NN_mae, RF_mae]]
table = tabulate.tabulate(metric, tablefmt='html')

display(HTML(table))

Metric,Dummy Reg,Linear Reg,Ridge Reg,Lasso Reg,ElasticNet,Nearest Neighbor,Random Forest
R**2,-0.0,0.215,0.154,-0.0,0.074,0.468,0.714
MSE,1.497,1.176,1.266,1.497,1.387,0.796,0.429
RMSE,1.224,1.084,1.125,1.224,1.178,0.892,0.655
MAE,0.974,0.819,0.872,0.974,0.929,0.574,0.434


Attempts to derive feature importance or understand variable contribution for Random Forest and Nearest Neighbor failed due to exceeding available resources: cuML KernelExplainer, cuML PermutationExplainer, sklearn .feature_importances_, and cuML .get_detailed_text().  So, while Random Forest appears to be the most accurate model for regression, it is not possible with the currently available resources to understand and rank feature importance.

The default values for Lasso Regression (L1 regularization) create a model that appears to simply follow the mean values.  

The errors from the regression models appear to exceed practical usefulness.  Converting to a classification problem will more likely yeild results that are more useful.

# Classification Models

In [ ]:
# Create first train/test split for classification.  Automated folds from cross-validation are consuming too many resources and taking too long to process.
Xcat_train, Xcat_test, ycat_train, ycat_test = cuml.model_selection.train_test_split(X, y_cat, test_size=0.2, shuffle=True, random_state=42, stratify=y_cat)

## Logistic Regression Classification Model

In [ ]:
# Logistic Regression does not converge with default values.
# Logistic Regression (defaults are: fit_intercept=True, C=1, penalty='l2', max_iter=1000, tol=1e-4).
Log_model = cuml.LogisticRegression(fit_intercept=True, C=1, penalty='none', max_iter=10000, tol=5e-4)
Log_model.fit(Xcat_train, ycat_train)
y_pred = Log_model.predict(Xcat_test)

In [ ]:
Log_acc, Log_roc, Log_cp, Log_cpMean, Log_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 60.0%
ROC AUC: 0.844
Mean Accuracy per Class & Standard Deviation: 27.4% +/- 16.2%
[[63.3 24.  19.9 19.3 15.2 13.5]
 [16.8 22.3 17.3 16.8 16.6 12.3]
 [ 9.9 19.7 21.2 19.3 19.5 17. ]
 [ 6.  17.5 19.5 22.3 20.9 22.3]
 [ 3.  12.2 14.  14.4 17.4 17.2]
 [ 1.   4.2  8.1  7.9 10.4 17.7]]


## Nearest Neighbors Classification Model

In [ ]:
# Nearest Neighbors Classifier with default values: n_neighbors=5
NNclass_model = cuml.neighbors.KNeighborsClassifier()
NNclass_model.fit(Xcat_train, ycat_train)
y_pred = NNclass_model.predict(Xcat_test)

In [ ]:
NN_acc, NN_roc, NN_cp, NN_cpMean, NN_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 68.2%
ROC AUC: 1.472
Mean Accuracy per Class & Standard Deviation: 55.2% +/- 9.2%
[[74.6 24.2 12.6 10.5  9.4  6.6]
 [12.6 49.2 15.6  8.4  6.7  5.1]
 [ 6.4 15.4 48.9 15.9  7.5  6.3]
 [ 3.8  6.6 15.6 50.1 16.6  6.7]
 [ 1.9  3.4  5.3 12.4 50.7 17.5]
 [ 0.6  1.2  1.9  2.6  9.2 57.7]]


### Nearest Neighbor Classification, n_neighbors=10

In [ ]:
# Nearest Neighbors Classifier, n_neighbors=10
NNclass_model = cuml.neighbors.KNeighborsClassifier(n_neighbors=10)
NNclass_model.fit(Xcat_train, ycat_train)
y_pred = NNclass_model.predict(Xcat_test)

In [ ]:
NN_acc, NN_roc, NN_cp, NN_cpMean, NN_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 67.5%
ROC AUC: 1.351
Mean Accuracy per Class & Standard Deviation: 56.5% +/- 8.1%
[[72.  19.7 10.9  9.2  7.1  4.7]
 [13.8 50.  15.8  8.   5.2  4.2]
 [ 7.1 16.7 49.3 16.   7.5  5.7]
 [ 4.3  7.8 16.  51.3 16.6  6.8]
 [ 2.2  4.3  5.9 12.6 54.4 16.7]
 [ 0.7  1.5  2.1  2.9  9.2 61.9]]


### Nearest Neighbor Hyperparameter Search

In [ ]:
# Search for best n_neighbors hyperparameter.  Model parameters are not causing the system to crash.
accuracy_list=[]

# Searching Nearest Neighbors from 2 to 9.
for i in range(8):
  NNclass_model = cuml.neighbors.KNeighborsClassifier(n_neighbors=(i+2))
  NNclass_model.fit(Xcat_train, ycat_train)
  y_pred = NNclass_model.predict(Xcat_test)
  score = str(np.round(cuml.metrics.accuracy.accuracy_score(ycat_test, y_pred), 3) * 100)
  accuracy_list.append(score)

In [ ]:
# Print Accuracy Score for each n_neighbors
for i in range(8):
  print(f'n_neighbors={(i+2)}, accuracy score: {accuracy_list[i]}%')

n_neighbors=2, accuracy score: 67.7%
n_neighbors=3, accuracy score: 67.80000000000001%
n_neighbors=4, accuracy score: 68.2%
n_neighbors=5, accuracy score: 68.2%
n_neighbors=6, accuracy score: 68.10000000000001%
n_neighbors=7, accuracy score: 68.0%
n_neighbors=8, accuracy score: 67.80000000000001%
n_neighbors=9, accuracy score: 67.7%


**SUMMARY** of NN: In a survey of n_neighbors from 2 to 10, the accuracy score peaked at n_neighbors=5 (default) with an overeall accuracy score of 68.3%.

## Random Forest Classification Model

In [ ]:
# Default RF model: n_estimators=100, max_depth=16
RFclass_model = cuml.ensemble.RandomForestClassifier()
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 64.7%
ROC AUC: 0.963
Mean Accuracy per Class & Standard Deviation: 65.0% +/- 9.1%
[[65.2  8.7  5.   4.5  0.8  0.5]
 [16.  60.1 11.8  6.1  1.2  1.4]
 [ 9.3 16.8 55.9 14.   2.7  2.3]
 [ 5.8  7.3 17.4 55.6 10.9  3.4]
 [ 2.9  4.8  7.1 16.3 72.2 11.5]
 [ 0.9  2.3  2.8  3.4 12.1 80.8]]


In [ ]:
# Random Forest Classifier Model, MAX_DEPTH = 100
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 76.7%
ROC AUC: 1.718
Mean Accuracy per Class & Standard Deviation: 73.9% +/- 5.6%
[[78.8 10.4  2.9  2.5  0.9  0.3]
 [10.8 71.9 12.7  3.1  0.9  0.4]
 [ 5.1 12.7 68.1 14.1  1.9  0.8]
 [ 3.1  3.  13.1 67.9 14.4  1.6]
 [ 1.6  1.6  2.5 11.1 73.4 13.5]
 [ 0.5  0.5  0.7  1.3  8.4 83.5]]


#### Random Forest Classifier, max_depth=100, n_estimators=200

In [ ]:
# Random Forest Classifier Model, MAX_DEPTH = 100, n_estimators = 200
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=200)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 76.9%
ROC AUC: 1.72
Mean Accuracy per Class & Standard Deviation: 74.4% +/- 5.4%
[[78.8 10.   2.7  2.3  0.7  0.3]
 [10.8 73.  12.7  3.1  0.9  0.3]
 [ 5.1 12.4 68.6 13.9  1.9  0.8]
 [ 3.2  2.7 12.9 68.4 14.   1.7]
 [ 1.6  1.4  2.4 11.2 74.  13.5]
 [ 0.5  0.4  0.7  1.1  8.5 83.4]]


#### Random Forest Classifier, max_depth=100, n_estimators=50

In [ ]:
# Random Forest Classifier Model, MAX_DEPTH = 100, n_estimators = 50
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=50)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 76.3%
ROC AUC: 1.72
Mean Accuracy per Class & Standard Deviation: 72.9% +/- 6.0%
[[78.9 11.1  3.3  2.7  1.1  0.4]
 [10.9 69.8 13.1  3.5  1.1  0.5]
 [ 5.1 13.1 66.5 14.4  2.3  0.8]
 [ 3.1  3.5 13.3 66.9 14.4  1.9]
 [ 1.6  1.9  2.8 11.2 72.7 13.9]
 [ 0.5  0.6  0.9  1.3  8.4 82.4]]


#### Random Forest Classifier, max_depth=16(default), n_estimators=300

In [ ]:
# Random Forest Classifier Model, MAX_DEPTH = 16, n_estimators = 300
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=16, n_estimators=300)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 64.8%
ROC AUC: 0.957
Mean Accuracy per Class & Standard Deviation: 66.0% +/- 9.0%
[[65.1  8.4  4.4  4.3  0.7  0.4]
 [16.  61.8 11.5  5.7  1.1  1.3]
 [ 9.3 16.4 57.5 13.8  2.1  2.1]
 [ 5.8  6.6 17.1 56.5 10.7  2.8]
 [ 2.9  4.6  6.8 16.5 72.8 11.3]
 [ 0.9  2.2  2.6  3.3 12.5 82.1]]


#### Random Forest Classifier, max_depth=100, n_estimators=300

In [ ]:
# Random Forest Classifier Model, MAX_DEPTH = 100, n_estimators = 300
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=300)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 77.0%
ROC AUC: 1.72
Mean Accuracy per Class & Standard Deviation: 74.6% +/- 5.3%
[[78.8  9.9  2.6  2.2  0.7  0.3]
 [10.8 73.5 12.7  3.   0.8  0.3]
 [ 5.1 12.3 68.9 13.8  1.8  0.8]
 [ 3.2  2.6 12.8 68.7 13.9  1.6]
 [ 1.6  1.3  2.3 11.1 74.3 13.3]
 [ 0.5  0.4  0.7  1.1  8.4 83.7]]


#### Random Forest Classifier, max_depth=100, n_estimators=350

In [ ]:
# Random Forest Classifier Model, MAX_DEPTH = 100, n_estimators = 350
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=350)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 77.0%
ROC AUC: 1.719
Mean Accuracy per Class & Standard Deviation: 74.7% +/- 5.3%
[[78.8  9.9  2.5  2.2  0.7  0.3]
 [10.8 73.6 12.6  3.   0.8  0.3]
 [ 5.1 12.2 69.1 13.9  1.7  0.8]
 [ 3.2  2.5 12.7 68.7 13.9  1.6]
 [ 1.6  1.3  2.3 11.2 74.4 13.2]
 [ 0.5  0.4  0.7  1.1  8.5 83.8]]


#### Random Forest Classifier, max_depth=150, n_estimators=300

In [ ]:
# Random Forest Classifier Model, MAX_DEPTH = 150, n_estimators = 300
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=150, n_estimators=300)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 77.0%
ROC AUC: 1.72
Mean Accuracy per Class & Standard Deviation: 74.6% +/- 5.3%
[[78.8  9.9  2.6  2.2  0.7  0.3]
 [10.8 73.5 12.7  3.   0.8  0.3]
 [ 5.1 12.3 68.9 13.8  1.8  0.8]
 [ 3.2  2.6 12.8 68.7 13.9  1.6]
 [ 1.6  1.3  2.3 11.1 74.3 13.3]
 [ 0.5  0.4  0.7  1.1  8.4 83.7]]


#### Random Forest Classifier: max_depth=200, n_estimators=300, split_criterion=gini(default)

In [ ]:
# Random Forest model: split_criterion=entropy
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=200, n_estimators=300, split_criterion=1)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 76.9%
ROC AUC: 1.716
Mean Accuracy per Class & Standard Deviation: 74.8% +/- 5.1%
[[78.5  9.7  2.6  2.3  0.9  0.3]
 [11.  74.1 12.7  2.9  1.   0.4]
 [ 5.2 12.  69.6 13.8  1.9  0.8]
 [ 3.2  2.5 12.4 68.9 13.7  1.6]
 [ 1.6  1.2  2.2 11.1 74.2 13.4]
 [ 0.4  0.3  0.6  1.   8.2 83.6]]


#### Random Forest Classifier, max_depth=200, n_estimators=300, split_criterion=entropy

In [ ]:
# Random Forest model: split_criterion=entropy
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=200, n_estimators=300, split_criterion=1)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 76.9%
ROC AUC: 1.716
Mean Accuracy per Class & Standard Deviation: 74.8% +/- 5.1%
[[78.5  9.7  2.6  2.3  0.9  0.3]
 [11.  74.1 12.7  2.9  1.   0.4]
 [ 5.2 12.  69.6 13.8  1.9  0.8]
 [ 3.2  2.5 12.4 68.9 13.7  1.6]
 [ 1.6  1.2  2.2 11.1 74.2 13.4]
 [ 0.4  0.3  0.6  1.   8.2 83.6]]


# Cross Validation of Optimized Random Forest Model.

## Multiple Random States Validation


In [ ]:
# CRASHING SYSTEM ON THIRD ITERATION - EXCEEDS AVAILABLE RAM.

# Optimal Random Forest model within the processing and resource constraints is max_depth=100, n_estimators=300
# Confirming results through multiple iterations of different random_states.

random = [47, 103, 333, 253, 66, 31]

acc_list = []
# roc_list = []
# cp_list = []
# cpMean_list = []
# cpSTD_list = []

# Run through the random numbers.
for state in random:
  
  # Create the Train/Test Split for each random number.
  Xcat_train, Xcat_test, ycat_train, ycat_test = cuml.model_selection.train_test_split(X, y_cat, test_size=0.2, shuffle=True, random_state=state, stratify=y_cat)

  RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=300, split_criterion=1)
  RFclass_model.fit(Xcat_train, ycat_train)
  y_pred = RFclass_model.predict(Xcat_test)
  y_pred = y_pred.astype(int)

#   RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

  accuracy = str(np.round(cuml.metrics.accuracy.accuracy_score(ycat_test, y_pred), 3) * 100)
  print(f'Random State: {state}, Accuracy: {accuracy}%')

  acc_list.append(accuracy)
#   roc_list.append(RF_roc)
#   cp_list.append(RF_cp)
#   cpMean_list.append(RF_cpMean)
#   cpSTD_list.append(RF_cpSTD)

  del RFclass_model

Random State: 47, Accuracy: 76.8%


In [ ]:
# Print Accuracy Score for each Random State
print(f'Accuracy List: {acc_list}')
print(f'Accuracy Mean: {np.mean(acc_list)}')
print(f'Accurcay Mean STD: {np.std(acc_list)}')

### Random Forest Classifier, max_depth=100, n_estimators=300, Train/Test Split random=47

In [ ]:
# Create first train/test split for classification.  Automated folds from cross-validation are consuming too many resources and taking too long to process.
Xcat_train, Xcat_test, ycat_train, ycat_test = cuml.model_selection.train_test_split(X, y_cat, test_size=0.2, shuffle=True, random_state=47, stratify=y_cat)

In [ ]:
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=300)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 76.9%
ROC AUC: 1.715
Mean Accuracy per Class & Standard Deviation: 74.2% +/- 5.1%
[[78.8 10.1  2.6  2.2  0.9  0.4]
 [10.8 72.9 13.1  2.9  0.8  0.4]
 [ 5.1 12.4 68.9 14.2  1.8  0.7]
 [ 3.2  2.7 12.5 68.2 14.   1.6]
 [ 1.6  1.4  2.2 11.4 74.1 14.2]
 [ 0.5  0.4  0.7  1.1  8.4 82.5]]


### Random Forest Classifier, max_depth=100, n_estimators=300, Train/Test Split random=103

In [ ]:
# Create first train/test split for classification.  Automated folds from cross-validation are consuming too many resources and taking too long to process.
Xcat_train, Xcat_test, ycat_train, ycat_test = cuml.model_selection.train_test_split(X, y_cat, test_size=0.2, shuffle=True, random_state=103, stratify=y_cat)

In [ ]:
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=300)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 77.0%
ROC AUC: 1.718
Mean Accuracy per Class & Standard Deviation: 74.6% +/- 4.9%
[[78.8  9.6  2.6  2.1  0.8  0.5]
 [10.8 73.7 12.8  2.9  0.9  0.3]
 [ 5.1 12.3 69.2 13.9  1.6  0.9]
 [ 3.2  2.6 12.4 68.8 14.1  1.8]
 [ 1.6  1.4  2.3 11.3 74.5 13.8]
 [ 0.5  0.4  0.7  1.   8.1 82.7]]


### Random Forest Classifier, max_depth=100, n_estimators=300, Train/Test Split random=33

In [ ]:
# Create first train/test split for classification.  Automated folds from cross-validation are consuming too many resources and taking too long to process.
Xcat_train, Xcat_test, ycat_train, ycat_test = cuml.model_selection.train_test_split(X, y_cat, test_size=0.2, shuffle=True, random_state=33, stratify=y_cat)

In [ ]:
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=300)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 77.0%
ROC AUC: 1.718
Mean Accuracy per Class & Standard Deviation: 74.7% +/- 5.3%
[[78.8  9.9  2.4  2.   0.9  0.3]
 [10.9 73.4 13.   2.9  0.7  0.3]
 [ 5.1 12.3 69.  14.1  1.8  0.4]
 [ 3.2  2.6 12.7 68.9 14.1  1.7]
 [ 1.6  1.3  2.3 11.2 74.2 13.3]
 [ 0.5  0.4  0.6  1.   8.3 84. ]]


### Random Forest Classifier, max_depth=100, n_estimators=300, Train/Test Split random=253

In [ ]:
# Create first train/test split for classification.  Automated folds from cross-validation are consuming too many resources and taking too long to process.
Xcat_train, Xcat_test, ycat_train, ycat_test = cuml.model_selection.train_test_split(X, y_cat, test_size=0.2, shuffle=True, random_state=253, stratify=y_cat)

In [ ]:
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=300)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 76.9%
ROC AUC: 1.715
Mean Accuracy per Class & Standard Deviation: 74.5% +/- 5.2%
[[78.8  9.8  2.6  2.1  1.   0.3]
 [10.9 73.4 12.9  2.7  0.9  0.3]
 [ 5.1 12.4 69.1 14.4  1.9  0.5]
 [ 3.2  2.6 12.6 68.4 13.6  1.5]
 [ 1.6  1.4  2.1 11.3 74.3 14.2]
 [ 0.5  0.4  0.7  1.   8.2 83.2]]


### Random Forest Classifier, max_depth=100, n_estimators=300, Train/Test Split random=66

In [ ]:
# Create first train/test split for classification.  Automated folds from cross-validation are consuming too many resources and taking too long to process.
Xcat_train, Xcat_test, ycat_train, ycat_test = cuml.model_selection.train_test_split(X, y_cat, test_size=0.2, shuffle=True, random_state=66, stratify=y_cat)

In [ ]:
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=300)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 76.9%
ROC AUC: 1.713
Mean Accuracy per Class & Standard Deviation: 74.3% +/- 5.0%
[[78.7 10.   2.7  2.3  1.   0.4]
 [10.9 73.3 12.7  2.7  0.8  0.4]
 [ 5.1 12.4 69.2 14.1  1.7  0.6]
 [ 3.2  2.5 12.5 68.4 14.1  1.4]
 [ 1.6  1.3  2.2 11.5 73.8 14.7]
 [ 0.5  0.4  0.7  1.1  8.6 82.4]]


### Random Forest Classifier, split_criterion=entropy, max_depth=100, n_estimators=300, Train/Test Split random=66

In [ ]:
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=300, split_criterion=1)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 76.8%
ROC AUC: 1.711
Mean Accuracy per Class & Standard Deviation: 74.4% +/- 4.7%
[[78.4  9.8  2.7  2.3  1.   0.4]
 [11.1 74.  12.6  2.9  1.1  0.5]
 [ 5.2 12.2 69.7 14.   2.1  0.6]
 [ 3.2  2.4 12.2 68.6 14.   1.6]
 [ 1.6  1.2  2.1 11.3 73.4 14.6]
 [ 0.5  0.3  0.6  1.   8.4 82.3]]


### Random Forest Classifier, split_criterion=entropy, max_depth=100, n_estimators=300, Train/Test Split random=33

In [ ]:
# Create first train/test split for classification.  Automated folds from cross-validation are consuming too many resources and taking too long to process.
Xcat_train, Xcat_test, ycat_train, ycat_test = cuml.model_selection.train_test_split(X, y_cat, test_size=0.2, shuffle=True, random_state=33, stratify=y_cat)

In [ ]:
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=200, n_estimators=300, split_criterion=1)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 76.8%
ROC AUC: 1.714
Mean Accuracy per Class & Standard Deviation: 74.8% +/- 5.0%
[[78.5  9.9  2.5  2.2  0.9  0.3]
 [11.1 73.9 12.9  2.9  0.9  0.4]
 [ 5.2 12.1 69.5 13.8  1.9  0.5]
 [ 3.2  2.4 12.4 69.  14.1  1.6]
 [ 1.6  1.3  2.1 11.2 74.1 13.6]
 [ 0.4  0.4  0.6  0.9  8.1 83.5]]


Changing the split_critereon from the default gini to entropy does not appear to increase accuracy in a signficant manner.

## **Summary of Validation.**

In [ ]:
# Mean and STD of accuracy score.
acc_list = [76.9, 77.0, 77.0, 76.9, 76.9]

# Print Accuracy Score for each Random State
# print(f'Accuracy List: {acc_list}')
print(f'Accuracy Mean: {str(np.round(np.mean(acc_list), 3))}%')
print(f'Accurcay Mean STD: {str(np.round(np.std(acc_list), 3))}%')

Accuracy Mean: 76.94%
Accurcay Mean STD: 0.049%


## KFold Cross Validation

In [ ]:
# CRASHES SYSTEM DUE TO EXCEEDING AVAILABLE RAM.

# KFold Stratified Cross validation.  Need to convert to Numpy array in order to use cross_val_score.
X = X.to_numpy()
y_cat = y_cat.to_numpy()

# Create 5-folds for cross validation
cv_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=33)

# Run the model with the appropriate data types.
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=300, output_type='numpy')
cv_acc = cross_val_score(RFclass_model, X, y_cat, cv=cv_fold)
print(f'RFclass_model: CV accuracy = {cv_acc.mean()} (std={cv_acc.std()})')

# Shapley Values

In [ ]:
# Create train/test split for model explainability.
Xcat_train, Xcat_test, ycat_train, ycat_test = cuml.model_selection.train_test_split(X, y_cat, test_size=0.2, shuffle=True, random_state=33, stratify=y_cat)

In [ ]:
# CRASHES SYSTEM DUE TO EXCEEDING AVAILABLE RAM.

# RAPIDS SHAP model explainability. 
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=200, n_estimators=300)
RFclass_model.fit(Xcat_train, ycat_train)

cu_explainer = cuml.explainer.KernelExplainer(
    model=RFclass_model.predict,
    data=Xcat_train,
    is_gpu_model=True,
    random_state=33,
    dtype=np.float32,
    output_type='cupy')

cu_shap_values = cu_explainer.shap_values(Xcat_test)
cu_shap_values  

# **Conclusions**

## **Regression Model Summary**.

In [ ]:
# Table Summarizing metrics from the different regression models.
metric = [['Metric', 'Dummy Reg', 'Linear Reg', 'Ridge Reg', 'Lasso Reg', 'ElasticNet', 'Nearest Neighbor', 'Random Forest'],
         ['R**2', Dum_r2, Lin_r2, Ridge_r2, Lasso_r2, EN_r2, NN_r2, RF_r2],
         ['MSE', Dum_mse, Lin_mse, Ridge_mse, Lasso_mse, EN_mse, NN_mse, RF_mse],
         ['RMSE', Dum_rmse, Lin_rmse, Ridge_rmse, Lasso_rmse, EN_rmse, NN_rmse, RF_rmse],
         ['MAE', Dum_mae, Lin_mae, Ridge_mae, Lasso_mae, EN_mae, NN_mae, RF_mae]]
table = tabulate.tabulate(metric, tablefmt='html')

display(HTML(table))

Metric,Dummy Reg,Linear Reg,Ridge Reg,Lasso Reg,ElasticNet,Nearest Neighbor,Random Forest
R**2,-0.0,0.215,0.154,-0.0,0.074,0.468,0.714
MSE,1.497,1.176,1.266,1.497,1.387,0.796,0.429
RMSE,1.224,1.084,1.125,1.224,1.178,0.892,0.655
MAE,0.974,0.819,0.872,0.974,0.929,0.574,0.434


## **Classification Model Summary.**

In [ ]:
metric = [['Metric','Logistic Regression', 'Nearest Neighbor', 'Random Forest'],
          ['ROC AUC', '0.844', '1.472', '1.72'],
          ['Total Accuracy', '60.0%', '68.2%', '77.0%'],
          ['Mean Accuracy per Class', '27.4%', '55.2%', '74.6%'],
          ['Accuracy per class STD', '16.2%', '9.2%', '5.3%']]
table = tabulate.tabulate(metric, tablefmt='html')

display(HTML(table))

Metric,Logistic Regression,Nearest Neighbor,Random Forest
ROC AUC,0.844,1.472,1.72
Total Accuracy,60.0%,68.2%,77.0%
Mean Accuracy per Class,27.4%,55.2%,74.6%
Accuracy per class STD,16.2%,9.2%,5.3%


## **Best Model: Random Forest (max_depth=100, n_estimators=300)**

In [ ]:
# Mean and STD of accuracy score.
acc_list = [76.9, 77.0, 77.0, 76.9, 76.9]

# Print Accuracy Score for each Random State
# print(f'Accuracy List: {acc_list}')
print(f'Total Accuracy Mean: {str(np.round(np.mean(acc_list), 3))}%')
print(f'Total Accurcay Mean STD: {str(np.round(np.std(acc_list), 3))}%')

Total Accuracy Mean: 76.94%
Total Accurcay Mean STD: 0.049%


In [ ]:
# Accuracy for each Drought Score Category.
metric = [['Class', 'Accuracy'],
          ['0', '78.8%'],
          ['1', '73.5%'],
          ['2', '68.9%'],
          ['3', '68.7%'],
          ['4', '74.3%'],
          ['5', '83.7%']]
table = tabulate.tabulate(metric, tablefmt='html')

display(HTML(table))

Class,Accuracy
0,78.8%
1,73.5%
2,68.9%
3,68.7%
4,74.3%
5,83.7%


**Conclusion:** The regression models do not give a good enough accuracy for practical implementation since the best Mean Absolute Error is 0.434 and a +/- of 0.434 does not give a good enough separation between the underlying Drought Score Categories.  Therefore, the modeling switched to classification.  The Random Forest Classification model gives a much more practical implementation for this project with an overall accuracy of 77% and a mean per class accuracy of 74.6 +/- 5.3%.